In [73]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from glob import glob
import torch
import scipy.io
import cv2
import random

In [3]:
skeleton_data = np.load('../data/skeleton_k10_movements.npz')
skeleton_classes, skeleton_mov = skeleton_data['arr_0'], skeleton_data['arr_1']

In [26]:
skeleton_mov.shape

(180, 60, 36)

In [27]:
sk1 = skeleton_mov[0]
sk1.shape

(60, 36)

In [28]:
sk1.min()

-0.5849068760871887

In [30]:
sk1.max()

0.9162541031837463

---

In [97]:
label_map = {
    1: 'swipe left',
    2: 'swipe right',
    3: 'wave',
    4: 'clap',
    5: 'throw',
    6: 'cross arms',
    7: 'basketball shoot',
    8: 'draw x',
    9: 'draw circle clockwise',
    10: 'draw circle counter clockwise',
    11: 'draw triangle',
    12: 'bowling',
    13: 'boxing',
    14: 'baseball swing',
    15: 'tennis swing',
    16: 'arm curl',
    17: 'tennis serve',
    18: 'two hand push',
    19: 'knock',
    20: 'catch',
    21: 'pick up then throw',
    22: 'jogging in place',
    23: 'walking in place',
    24: 'sit to stand',
    25: 'stand to sit',
    26: 'lunge',
    27: 'squat'
}

In [9]:
# for p in glob(f'../data/UTD-MHAD/'):
# file_name = p.split('\\')[-1]
# action, subject, time, _ = file_name.split('_')
p = '../data/UTD-MHAD/UTD-MHAD-Skeleton/a10_s1_t1_skeleton.mat'
mat = scipy.io.loadmat(p)
np_data = np.array(mat['d_skel'])

In [11]:
np_data.shape

(20, 3, 76)

In [14]:
cols = [4,8,5,9,6,10,12,16,13,17,14,18]
point_mapping = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]

In [15]:
skel_res = np_data[cols, :, :]

In [16]:
skel_res.shape

(12, 3, 76)

In [17]:
xyz_cords = np.transpose(skel_res, (2,0,1))
xyz_cords.shape

(76, 12, 3)

In [60]:
x_min = xyz_cords[:, :, 0].min()
y_min = xyz_cords[:, :, 1].min()
adder = [-1*x_min, -1*y_min, 0]
xyz_coords = xyz_cords * [1,-1,1]
xyz_coords = xyz_coords + adder
x_max = xyz_coords[:, :, 0].max()
y_max = xyz_coords[:, :, 1].max()
z_max = xyz_coords[:, :, 2].max()
divs = [x_max, y_max, z_max]
xyz_coords = xyz_coords/divs
xyz_coords.shape

(76, 12, 3)

In [51]:
xyz_coords[:, :, 0].min()

0.0

In [18]:
h, w = 128, 64

In [20]:
def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3

def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for frame in points:
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

In [61]:
gen_video(xyz_coords, '../tmp/sample_utd_skel.mp4', 512, 512)

In [58]:
xyz_coords[:, :, 0].max()

1.0

In [91]:
def random_padding(com_arr, output_size):
    # pad and interpolate
    arr_df = pd.DataFrame(data=com_arr, columns=[str(i) for i in range(com_arr.shape[1])])
    curr_size = arr_df.shape[0]
    fill_points = random.sample(range(0, curr_size), output_size-curr_size)

    df_data = []
    for i, row in arr_df.iterrows():
        df_data.append(row.values)
        if i in fill_points:
            df_data.append(pd.Series(dtype='float64').values)

    df_new = pd.DataFrame(df_data)
    df_new = df_new.interpolate(method='linear', limit_direction='forward', axis=0)
    return df_new.values 

def random_sampling(com_arr, output_size, skip_pre=0, skip_post=0):
    curr_size = com_arr.shape[0]
    sample_points = random.sample(range(skip_pre, curr_size-skip_post), output_size)
    sample_points.sort()
    out_arr = com_arr[sample_points, :]
    return out_arr

In [100]:
def post_process(arr, seq_len):
    cols = [4,8,5,9,6,10,12,16,13,17,14,18]
    skel_res = arr[cols, :, :]
    xyz_cords = np.transpose(skel_res, (2,0,1))
    x_min = xyz_cords[:, :, 0].min()
    y_min = xyz_cords[:, :, 1].min()
    adder = [-1*x_min, -1*y_min, 0]
    xyz_coords = xyz_cords * [1,-1,1]
    xyz_coords = xyz_coords + adder
    x_max = xyz_coords[:, :, 0].max()
    y_max = xyz_coords[:, :, 1].max()
    z_max = xyz_coords[:, :, 2].max()
    divs = [x_max, y_max, z_max]
    xyz_coords = xyz_coords/divs

    xyz_coords = xyz_coords.reshape(xyz_coords.shape[0], -1)
    # print(xyz_coords.shape[0])
    # crop seq 
    if xyz_coords.shape[0] > seq_len:
        xyz_coords = random_sampling(xyz_coords, seq_len)
    else:
        xyz_coords = random_padding(xyz_coords, seq_len)
        
    # print(xyz_coords.shape)
    
    return xyz_coords

def readUTDFiles(root_path, n=60):
        data = []
        labels = []
        subjects = []
        
        for p in glob(f'{root_path}/*.mat'):
            file_name = p.split('\\')[-1]
            action, subject, time, _ = file_name.split('_')
            mat = scipy.io.loadmat(p)
            np_data = np.array(mat['d_skel'])
        
            data.append(post_process(np_data, n))
            labels.append(label_map[int(action.strip('a'))])
            subjects.append(subject)
        # print(f"data len : {np.array(data).shape}, data 0 shape : {data[0].shape}")
        return np.array(data), np.array(labels), np.array(subjects)

In [101]:
skel_movs, skel_classes, skel_ids = readUTDFiles('../data/UTD-MHAD/UTD-MHAD-Skeleton/')

In [102]:
skel_movs.shape

(861, 60, 36)

In [103]:
full_data = {'actions': skel_classes, 'sequences': skel_movs, 'id': skel_ids}
np.savez('../data/skeleton_UTD_V1.npz', skel_classes, skel_movs, skel_ids)

In [ ]:
skel_classes

---